## A working example of the postupdateprocesses function: `secfsdstools.x_examples.automation.automation.define_extra_processes` (introduced in 1.8.0)

<span style="color: #FF8C00;">==========================================================</span>

**If you find this tool useful, a sponsorship would be greatly appreciated!**

**https://github.com/sponsors/HansjoergW**

<span style="color: #FF8C00;">==========================================================</span>

### What this pipeline creates

It result in creating the following bags:

- a single joined bag per statement (BS, IS, CF, ..) that will contain the data from all available quarters.
- standardized bags for BS, IS, CF that contain data from all the available quarters.
- a single joined bag containing all the data from all statements from all available quarters.

Moreover, all these bags are updated in an efficient way, as soon as new data becomes available at the SEC website.

Note: especially the creation of the final single joined bag is quite memory intensive. (Have a look at notebook 08_02_automation_a_memory_optimized_example for a version that needs less memory)


### How to use the example

The package `secfsdstools.x_examples.automation` provides a default implemention of a postupdateprocesses function: `define_extra_processes`.

You can use this function directly by adding it to your configuration file together with some additional configuration parameters used by it: 
<pre>
[DEFAULT]
...
postupdateprocesses=secfsdstools.x_examples.automation.automation.define_extra_processes

[Filter]
filtered_dir_by_stmt_joined = C:/data/sec/automated/_1_filtered_by_stmt_joined

[Concat]
concat_dir_by_stmt_joined = C:/data/sec/automated/_2_concat_by_stmt_joined

[Standardizer]
standardized_dir = C:/data/sec/automated/_3_standardized

; [SingleBag]
; singlebag_dir = C:/data/sec/automated/_4_single_bag
</pre>

The function will add 3 additional steps and a fourth optional step. The optional step is only executed if the needed parameter `singlebag_dir` is defined. 

These steps add the following processing:

The first step creates a joined bag for every zip file which is filtered for 10-K and 10-Q reports only
and also applies the filters `ReportPeriodRawFilter`, `MainCoregRawFilter`, `USDOnlyRawFilter`, `OfficialTagsOnlyRawFilter`. 
Furthermore, the data is also split by stmt.
The filtered joined bag is stored under the path that is defined under `filtered_dir_by_stmt_joined` in the configuration file.
The resulting directory structure will look like this:


    <filtered_dir_by_stmt_joined>
        quarter
            2009q2.zip
                BS
                CF
                CI
                CP
                EQ
                IS
            ...

The second step creates a single joined bag for every statement (balance sheet, income statement,
cash flow, cover page, ...) that contains the data from all zip files, resp from all the
available quarters. These bags are stored under the path defined as `concat_dir_by_stmt_joined`.
The resulting directory structure will look like this:

    <concat_dir_by_stmt_joined>
        BS
        CF
        CI
        CP
        EQ
        IS    


The third step standardizes the data for balance sheet, income statement, and cash flow and stores
the standardized bags under the path that is defined as `standardized_dir`.
The resulting structure will look like this:

    <standardized_dir>
        BS
        CF
        IS    
    

The fourth step is optional and is only executed if the configuration file contains an entry
for `singlebag_dir`. If it does, it will create a single joined bag concatenating all the bags
created in the second step, so basically creating a single bag that contains all the filtered data from
all the available zip files, resp. quarters. 
In framework versions prior to version 2.1, this step needed quite a lot of memory. This was improved 
in version 2.1, which does the concatenation directly on the filesystem, without loading the data into
memory and hence has a very low memory footproint.
The resulting directory structure will look like this:

    <singlebag_dir>
        all


Hint -> data can directly be loaded with the JoinedDataBag load, resp with StandardizedBag load.


### How the example is implemented.

Let us have a look at the implementation of the the function `define_extra_processes`:


In [1]:
from typing import List

from secfsdstools.a_config.configmodel import Configuration
from secfsdstools.c_automation.task_framework import AbstractProcess

# Note: that we are using a few basic implementations of the g_pipeline package
from secfsdstools.g_pipelines.concat_process import ConcatByNewSubfoldersProcess, \
    ConcatByChangedTimestampProcess
from secfsdstools.g_pipelines.filter_process import FilterProcess
from secfsdstools.g_pipelines.standardize_process import StandardizeProcess


def define_extra_processes(configuration: Configuration) -> List[AbstractProcess]:

    # first, we read the configuration parameters. 
    joined_by_stmt_dir = configuration.config_parser.get(section="Filter",
                                                         option="filtered_dir_by_stmt_joined")

    concat_by_stmt_dir = configuration.config_parser.get(section="Concat",
                                                         option="concat_dir_by_stmt_joined")

    standardized_dir = configuration.config_parser.get(section="Standardizer",
                                                       option="standardized_dir")

    # note that the single bag dir is optional and therefore has a fallback value of ""
    singlebag_dir = configuration.config_parser.get(section="SingleBag",
                                                    option="singlebag_dir",
                                                    fallback="")

    
    processes: List[AbstractProcess] = []

    # The first step filters the data. It is apllied on the data of every available transformed parquet folder.
    # If nothing else is configured, it will filter for 10-K and 10-Q reports only.
    # Moreover, it will also apply the filters ReportPeriodRawFilter, MainCoregRawFilter, USDOnlyRawFilter, and OfficialTagsOnlyRawFilter. 
    # You can actually configure wether you want the data to be saved as RawDataBag or a JoinedDataBag.
    # In our case, we will use the JoinedDataBag.
    # As another parameter, we can configure that the data is split up by stmt. So the data for every statement is saved in its on subfolder.
    # Therefore, the result will be a folder for every quarter containing subfolders for every statement (BS, CF, CI, CP, EQ, and IS).
    # Note that the execution processed in parallel.
    processes.append(
        # 1. Filter, join, and save by stmt
        FilterProcess(db_dir=configuration.db_dir,
                      target_dir=joined_by_stmt_dir,
                      bag_type="joined",
                      save_by_stmt=True,
                      execute_serial=configuration.no_parallel_processing
                      )
    )

    # Next, we want to create a single JoinedBag for every stmt that should contain the data from all quarters.
    # To do that, we can use the ConcatByNewSubfoldersProcess. 
    # It will load the joined bags from the appropriate subfolders inside the root_dir, concat them into single 
    # bag that is then stored in the target_dir.
    # Note: This AbstractProcess implementation stores a metainf file in the target folder which contains a list
    # of all the subfolders from the root-dir, that are already concatenated into the target. Using that file,
    # it will only be executed, if a new subfolder appears in the root_dir.
    # After these steps, we will have a joined bag for every statement (BS, CF, IS, ...) containing the data from all
    # available quarters.
    processes.extend([
        # 2. building datasets with all entries by stmt
        ConcatByNewSubfoldersProcess(root_dir=f"{joined_by_stmt_dir}/quarter",
                                     target_dir=f"{concat_by_stmt_dir}/BS",
                                     pathfilter="*/BS"
                                     ),
        ConcatByNewSubfoldersProcess(root_dir=f"{joined_by_stmt_dir}/quarter",
                                     target_dir=f"{concat_by_stmt_dir}/CF",
                                     pathfilter="*/CF"
                                     ),
        ConcatByNewSubfoldersProcess(root_dir=f"{joined_by_stmt_dir}/quarter",
                                     target_dir=f"{concat_by_stmt_dir}/CI",
                                     pathfilter="*/CI"
                                     ),
        ConcatByNewSubfoldersProcess(root_dir=f"{joined_by_stmt_dir}/quarter",
                                     target_dir=f"{concat_by_stmt_dir}/CP",
                                     pathfilter="*/CP"
                                     ),
        ConcatByNewSubfoldersProcess(root_dir=f"{joined_by_stmt_dir}/quarter",
                                     target_dir=f"{concat_by_stmt_dir}/EQ",
                                     pathfilter="*/EQ"
                                     ),
        ConcatByNewSubfoldersProcess(root_dir=f"{joined_by_stmt_dir}/quarter",
                                     target_dir=f"{concat_by_stmt_dir}/IS",
                                     pathfilter="*/IS"
                                     )
    ])

    # As a third step, we standardize the data for BS, CF, and IS. 
    # This can be done with the StandardizeProcess. As input it expects a folder that contains the subfolders BS, CF, and IS
    processes.append(
        # 3. Standardize the data
        StandardizeProcess(root_dir=f"{concat_by_stmt_dir}",
                           target_dir=standardized_dir),
    )

    # If the parameter singlebag_dir is configured, we use the ConcatByChangedTimestampProcess
    # to concat all the bags from the second step together into a single bag. 
    # This produces a single bag containing all the available data.
    # Note: this used a lot of memory prior to version 2.1
    # Note: this implementation stores a metainf file in the target folder, that contains the 
    # timestamp of the latest modifications inside the root_dir. Threfore, it will only be executed
    # if any file in the root_dir has a newer timestamp than the saved one.
    if singlebag_dir != "":
        # 4. create a single joined bag with all the data, if it is defined
        processes.append(
            ConcatByChangedTimestampProcess(
                root_dir=f"{concat_by_stmt_dir}/",
                target_dir=f"{singlebag_dir}/all",
            )
        )

    return processes


2025-02-05 06:20:25,256 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg


If you want to know more about the implementation details of the FilterProcess, ConcatByNewSubfoldersProcess, ConcatByChangedTimestampProcess, and StandardizeProcess, have a look at the comments in the code.